In [62]:
!gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: C:\Users\Mila\ElectricCarData.csv

  0%|          | 0.00/8.20k [00:00<?, ?B/s]
100%|##########| 8.20k/8.20k [00:00<?, ?B/s]


In [63]:
import pandas as pd

In [64]:
data = pd.read_csv('ElectricCarData.csv')

In [65]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


In [66]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [67]:
data['Brand'] = label_encoder.fit_transform(data['Brand'])
data['Model'] = label_encoder.fit_transform(data['Model'])
data['RapidCharge'] = label_encoder.fit_transform(data['RapidCharge'])
data['PowerTrain'] = label_encoder.fit_transform(data['PowerTrain'])
data['PlugType'] = label_encoder.fit_transform(data['PlugType'])
data['BodyStyle'] = label_encoder.fit_transform(data['BodyStyle'])
data['Segment'] = label_encoder.fit_transform(data['Segment'])

In [68]:
print(data)

     Brand  Model  AccelSec  TopSpeed_KmH  Range_Km  Efficiency_WhKm  \
0       30     46       4.6           233       450              161   
1       31     33      10.0           160       270              167   
2       23      0       4.7           210       400              181   
3        2    101       6.8           180       360              206   
4        9     78       9.5           145       170              168   
..     ...    ...       ...           ...       ...              ...   
98      20      6       7.5           160       330              191   
99       1     94       4.5           210       335              258   
100     20      8       5.9           200       325              194   
101     20     10       5.1           200       375              232   
102      3     42       7.5           190       400              238   

    FastCharge_KmH  RapidCharge  PowerTrain  PlugType  BodyStyle  Segment  \
0              940            1           0         2     

In [69]:
import numpy as np
data.replace('-', np.nan, inplace=True)

In [70]:
data.dropna(inplace=True)

In [71]:
data = data.apply(pd.to_numeric, errors='coerce')

In [72]:
from sklearn.model_selection import train_test_split

X = data.drop('PriceEuro', axis=1)  
y = data['PriceEuro']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (78, 13)
X_test shape: (20, 13)
y_train shape: (78,)
y_test shape: (20,)


In [73]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

tree_reg = DecisionTreeRegressor(max_depth=5, min_samples_split=2, max_features=None, random_state=42)

tree_reg.fit(X_train, y_train)

y_train_pred = tree_reg.predict(X_train)

train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
print("Training RMSE:", train_rmse)

Training RMSE: 4558.76311428678


In [74]:
y_test_pred = tree_reg.predict(X_test)
print("Predicted Outcomes on Test Set:", y_test_pred)

Predicted Outcomes on Test Set: [54772.8        65310.         89895.         50000.
 36418.14285714 71614.33333333 89895.         71614.33333333
 54772.8        65310.         37054.18181818 54772.8
 89895.         28887.41666667 28887.41666667 36418.14285714
 37054.18181818 36418.14285714 42188.33333333 36418.14285714]


In [75]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


mae = mean_absolute_error(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)
rmse = mean_squared_error(y_test, y_test_pred, squared=False)  
r2 = r2_score(y_test, y_test_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


Mean Absolute Error (MAE): 8259.790952380954
Mean Squared Error (MSE): 111105085.9859217
Root Mean Squared Error (RMSE): 10540.639733238286
R-squared (R2) Score: 0.8408744802780284


In [76]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

xgb_reg = XGBRegressor(
    n_estimators=100, 
    max_depth=3,       
    learning_rate=0.1,  
    objective='reg:squarederror',  
    random_state=42
)

xgb_reg.fit(X_train, y_train)

y_train_pred_xgb = xgb_reg.predict(X_train)

train_rmse_xgb = mean_squared_error(y_train, y_train_pred_xgb, squared=False)
print("Training RMSE (XGB):", train_rmse_xgb)


Training RMSE (XGB): 1343.18022756834


In [77]:
y_test_pred_xgb = xgb_reg.predict(X_test)

print("Predicted Outcomes on Test Set (XGBoost):", y_test_pred_xgb)

Predicted Outcomes on Test Set (XGBoost): [ 74432.9    93048.766  95322.02   37017.543  37174.395  67248.98
 112861.29   76343.54   61079.957  64652.008  40009.723  61756.98
 107958.63   33593.605  23952.104  34828.035  34866.26   35797.164
  38986.914  32171.5  ]


In [78]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae_xgb = mean_absolute_error(y_test, y_test_pred_xgb)
mse_xgb = mean_squared_error(y_test, y_test_pred_xgb)
rmse_xgb = mean_squared_error(y_test, y_test_pred_xgb, squared=False)  
r2_xgb = r2_score(y_test, y_test_pred_xgb)


print("Mean Absolute Error (MAE) - XGBoost:", mae_xgb)
print("Mean Squared Error (MSE) - XGBoost:", mse_xgb)
print("Root Mean Squared Error (RMSE) - XGBoost:", rmse_xgb)
print("R-squared (R2) Score - XGBoost:", r2_xgb)


Mean Absolute Error (MAE) - XGBoost: 4221.23369140625
Mean Squared Error (MSE) - XGBoost: 28797322.413887214
Root Mean Squared Error (RMSE) - XGBoost: 5366.313670844001
R-squared (R2) Score - XGBoost: 0.9587562634505172


In [80]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae_tree = mean_absolute_error(y_test, y_test_pred)
mse_tree = mean_squared_error(y_test, y_test_pred)
r2_tree = r2_score(y_test, y_test_pred)

mae_xgb = mean_absolute_error(y_test, y_test_pred_xgb)
mse_xgb = mean_squared_error(y_test, y_test_pred_xgb)
r2_xgb = r2_score(y_test, y_test_pred_xgb)

print("DecisionTreeRegressor Metrics:")
print("MAE:", mae_tree)
print("MSE:", mse_tree)
print("R2 Score:", r2_tree)
print()

print("XGBoostRegressor Metrics:")
print("MAE:", mae_xgb)
print("MSE:", mse_xgb)
print("R2 Score:", r2_xgb)


DecisionTreeRegressor Metrics:
MAE: 8259.790952380954
MSE: 111105085.9859217
R2 Score: 0.8408744802780284

XGBoostRegressor Metrics:
MAE: 4221.23369140625
MSE: 28797322.413887214
R2 Score: 0.9587562634505172
